In [1]:
def test(url):
    unique_url={}
    all_data = [
        {'page_time': 13.5,'url': 'google.com'},{'page_time': 56.5,'url': 'moat.com'},{'page_time': 56.5,'url': 'vip.com'}
    ]
    
    if url not in unique_url:
        unique_url[url] = 1
    else:
        unique_url[url] += 1

    for records in all_data:
        print(records['url'])
        if records['url'] == url:
            hitcount = unique_url[records['url']]
            newpair = {"total_hits": hitcount}
            records.update(newpair)
        records
    return all_data
        
def top_ten():
    val = test('google.com')
    return sorted(val[:1], key = lambda i: i['total_hits'], reverse=True)
        
#         for key,value in val.items():
#             if value == url and value in unique_url.keys():
#                 newpair = {"total_hits": unique_url.get(value)}
#         if url in val.values():
#             val.update(newpair)
#         else:
#             val
                
#     return all_data
    


In [2]:
print(top_ten())

google.com
moat.com
vip.com
[{'page_time': 13.5, 'url': 'google.com', 'total_hits': 1}]


In [3]:
lis = [{ "name" : "Nandini", "age" : 20},  
{ "name" : "Manjeet", "age" : 21 }, 
{ "name" : "Nikhil" , "age" : 19 }]

print(sorted(lis[:2], key = lambda i: i['age'], reverse = True))

[{'name': 'Manjeet', 'age': 21}, {'name': 'Nandini', 'age': 20}]


In [4]:
def test(url):
    myData = []
    hit = 0
    data = {'page_time': 13.5,'url': 'google.com'}
    if data['url'] == url:
        hit += 1
        data["hits"] = hit
    myData.append(data)
    return myData


In [5]:
print(test('google.com'))

[{'page_time': 13.5, 'url': 'google.com', 'hits': 1}]


In [6]:
#counter_ API

import flask
import cx_oracle
from flask import request, jsonify
from itertools import groupby
from operator import itemgetter

#we assume that the function to calculate the time spent on a url, the url visited and the function to check is_bot is already caulated and data stored in 
#in a table in Oracle DB called browserRecord

app = flask.Flask(__name__)
app.config["DEBUG"] = True

# impression = [
#     {
#       'url': 'moat.com',
#       'page_time': 234.2,
#       'total_hits': 1,
#       'is_bot': 1
#     },
#     {
#       'url': 'google.com',
#       'page_time': 467.2,
#       'total_hits': 1,
#       
#     },
#     {
#       'url': 'google.com',
#       'page_time': 34.2,
#       'total_hits': 1,
#       'is_bot': 0
#     }
# ]

#impression route validates the query parament and fetch the data from the database 
#It records a hit on very url and spits out a JSON structure of the data.

@app.route('/impression', methods=['GET'])
def api_filter():
    hit = 0
    is_bot = 0
    query_parameters = request.args

    page_time = query_parameters.get('page_time')
    url = query_parameters.get('url')
    # if the request paramenter is given in the argument then get it
    if query_parameters.get('is_bot'):
        is_bot = query_parameters.get('is_bot')
        
    #impression list to store updated rawData objects with hit record along with page_time and is_bot
    impression = []
    filterBy = []

    query = "SELECT * FROM bowserRecord WHERE"
    
    #check the query parameter and update query based on given parameter
    #whenever a url is queried record hit
    if url:
        query += 'url=? AND'
        filter.append[url]
        hit = 1
       
    if page_time:
        query += 'page_time=? AND'
        filter.append[page_time]
    if is_bot:
        query += 'is_bot=?'
        filter.append[is_bot]

    #connect to the Oracle DB and fetch result based on query parameter
    #convert result into json using python flask jsonify function
    #update the json object with url hit and append records to impression list
    conn = cx_Oracle.connect('DB')
    cur = con.cursor()
    result = cur.execute(query, filterBy).fetchall()
    rawData = jsonify(result)
    rawData["hit"] = hit
    impression.append(rawData)
    return impression



#aggregate function computes the aggregated data and returns the list of objects
def aggregate():
    data = api_filter()
    table = {}

    #create a hashmap called tale and aggregate by url for each object in data.
    #table structure {'url1': {url: 'url1', avg_page_time: sum_page_time, 'total_hits': sum_total_hits, 'bot_percentage': sum_isbot}, 'url2': {same structure as url1} }
    #aggData structure [{url: 'url1', 'avg_page_time': avg, 'total_hits': sum_total_hits, 'bot_percentage': %is_bot}, 'url2': {same structure as url1},{}]
    for obj in data:
        #put default values in table
        if obj['url'] not in table:
            table[obj['url']] = {'url': obj['url'], 'avg_page_time':0.0, 'total_hits': 0, 'bot_percentage': 0.0}
            
        table[obj['url']]['total_hits'] += obj['total_hits']
        table[obj['url']]['avg_page_time'] += obj['page_time']
        
        #check if is_bot is present in each object, then find the cummulative sum
        if obj['is_bot']: 
            table[obj['url']]['bot_percentage'] += obj['is_bot']
   
    aggData = list(table.values())
    
    # loop through each object in aggData, compute the avg_page_time and bot_percentage and update aggData with these 
    #new values
    for val in aggData:
        avg = float(val['avg_page_time']/val['total_hits'])
        bot = float(val['bot_percentage']/val['total_hits']) 
    val['avg_page_time'] = avg
    val['bot_percentage'] = bot
    
    return aggData


# top_ten route calls the agregate function, sorts and order the objects by total_hits in descending order
@app.route('/top_ten', methods=['GET'])
def top_ten():
    myData = aggregate()
    #return top 10 record using python list slice [:10] lambda function helps to sort by total hits
    #reverse set to true so it orders in decreasing order of total hits
    return sorted(myData[:10], key = lambda i: i['total_hits'], reverse=True) 


app.run()

    

    


ModuleNotFoundError: No module named 'cx_oracle'

In [28]:
print(agregate())

234.2
521.6
[{'url': 'moat.com', 'avg_page_time': 234.2, 'total_hits': 1, 'bot_percentage': 1.0}, {'url': 'google.com', 'avg_page_time': 173.86666666666667, 'total_hits': 3, 'bot_percentage': 0.02}]
